# Create Heat Map for significant Spliceosome genes

This notebook looks at the significant genes in at least one cancer in the DNA Replication pathway. Pancancer heat maps are created with circle size showing significance and color showing differences in median.

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats
import gseapy as gp
import re
import sys 

import cptac
import cptac.utils as u

import plot_utils as p

# Step 1: Run GSEA for significant genes in at least 1 cancer

First read in sig_pval_all_proteins.csv into a df. This csv file contains only genes with a significant p-value in at least one cancer. Then run GSEA using a list of genes from the df.

In [2]:
root = R'~\Github\WhenMutationsDontMatter\PTEN\Step_3_trans_effect\csv'
sig_df = pd.read_csv(root+R'\sig_pval_heatmap.csv')

prot_list = list(sig_df.Proteomics) # list of genes with a sig pval in >= 1 cancer
prot_enr = gp.enrichr(gene_list = prot_list, description='Tumor_partition', gene_sets='KEGG_2016', 
                       outdir='/Enrichr')

In [3]:
prot_enr.res2d.head()

,Term,Overlap,P-value,Adjusted P-value,Old P-value,Old Adjusted P-value,Odds Ratio,Combined Score,Genes,Gene_set
0,Spliceosome Homo sapiens hsa03040,88/134,2.936551e-44,8.604095e-42,0,0,4.860965,487.246193,RBM25;EIF4A3;DDX42;HNRNPU;PRPF19;USP39;EFTUD2;...,KEGG_2016
1,RNA transport Homo sapiens hsa03013,77/172,1.113872e-23,1.631823e-21,0,0,3.313652,175.131871,CYFIP2;NUP107;NUP188;RPP30;EIF4A3;PHAX;PNN;XPO...,KEGG_2016
2,DNA replication Homo sapiens hsa03030,23/36,3.613010e-12,3.528707e-10,0,0,4.729007,124.592696,RFC5;FEN1;RFC3;PCNA;RFC4;MCM7;RFC1;RFC2;PRIM1;...,KEGG_2016
3,mRNA surveillance pathway Homo sapiens hsa03015,38/91,2.801846e-11,2.052353e-09,0,0,3.090914,75.103503,SMG1;RBM8A;RNMT;CSTF3;EIF4A3;MSI1;PPP2R2A;MSI2...,KEGG_2016
4,Mismatch repair Homo sapiens hsa03430,15/23,1.473843e-08,8.636717e-07,0,0,4.827342,87.050531,RFC5;RFC3;PCNA;RFC4;RFC1;RFC2;RPA1;MLH1;POLD3;...,KEGG_2016


# Step 2: Get the list of significant genes 

In [4]:
index = 2
trans = prot_enr.res2d.Genes[index]
genes = trans.split(';')
print(prot_enr.res2d.Term[index])
print('total genes:',len(genes))

DNA replication Homo sapiens hsa03030
total genes: 23


# Step 3: Create HeatMap

Slice out genes from the DNA Replication pathway from the df with genes sig in > 1 cancer.

In [5]:
# sig > 1 cancer
bool_df = sig_df.Proteomics.isin(genes)
plot_df = sig_df[bool_df]
len(plot_df.Proteomics.unique())

23

In [6]:
p.plotCircleHeatMap(plot_df, circle_var = 'P_Value', color_var='Medians', x_axis= 'Proteomics', y_axis = 'Cancer',
                    plot_height=400, plot_width= 800)

C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\plot_utils\__init__.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["size2"] = df[circle_var].apply(lambda x: -1*(np.log(x)))
C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\plot_utils\__init__.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['size'] = (df["size2"])*3


Loading BokehJS ...

# Step 4: Create a HeatMap with both +/- differences in median

 Read in the pos_neg_df.csv to create a df with only genes that have a pos and neg difference in median in different cancers. Slice out the genes that have a pos and neg difference in median in the pathway using the list of genes with a significant p-value in the pathway. 

In [7]:
pos_neg_df = pd.read_csv(root+R'\pos_neg_df.csv')

In [8]:
get = pos_neg_df.Proteomics.isin(genes) # bool df where True has both pos and neg
genes_pn = pos_neg_df[get] # Keep only genes with pos and neg
genes_pn.Proteomics.unique()

array(['MCM6', 'MCM4', 'RFC5', 'MCM5', 'MCM2', 'MCM7', 'MCM3', 'RFC2',
       'RFC4', 'SSBP1', 'POLA2'], dtype=object)

In [9]:
p.plotCircleHeatMap(genes_pn, circle_var = 'P_Value', color_var='Medians', x_axis= 'Proteomics', y_axis = 'Cancer',
                    plot_height=400, plot_width=600)

Loading BokehJS ...

# Sig in multiple cancers

In [10]:
mult_sig_df = pd.read_csv(root+R'\mult_sig_pval_heatmap.csv')

mult_sig_list = list(mult_sig_df.Proteomics.unique()) # list of genes with a sig pval in > 1 cancer
enr2 = gp.enrichr(gene_list = mult_sig_list, description='Tumor_partition', gene_sets='KEGG_2016', 
                       outdir='/Enrichr')

In [11]:
enr2.res2d.head()

,Term,Overlap,P-value,Adjusted P-value,Old P-value,Old Adjusted P-value,Odds Ratio,Combined Score,Genes,Gene_set
0,Spliceosome Homo sapiens hsa03040,21/134,4.645154e-13,1.361030e-10,0,0,7.626103,216.564409,DDX5;NCBP1;NCBP2;DDX23;DDX42;THOC3;PRPF40A;THO...,KEGG_2016
1,DNA replication Homo sapiens hsa03030,12/36,3.876348e-12,5.678850e-10,0,0,16.220600,426.214559,RFC5;RFC3;PCNA;RFC4;MCM7;RFC1;RFC2;MCM3;MCM4;M...,KEGG_2016
2,RNA transport Homo sapiens hsa03013,19/172,2.855921e-09,2.789283e-07,0,0,5.375431,105.755548,RANBP2;EIF5B;NUP210;NUP155;NCBP1;NUP133;NCBP2;...,KEGG_2016
3,Ribosome biogenesis in eukaryotes Homo sapiens...,14/89,3.615167e-09,2.648110e-07,0,0,7.654665,148.792361,UTP6;IMP3;WDR3;HEATR1;NAT10;WDR75;IMP4;PWP2;UT...,KEGG_2016
4,Mismatch repair Homo sapiens hsa03430,8/23,1.112006e-08,6.516354e-07,0,0,16.925844,309.988622,RFC5;MSH6;RFC3;PCNA;RFC4;RFC1;MSH2;RFC2,KEGG_2016


In [12]:
index2 = 1
trans2 = enr2.res2d.Genes[index2]
genes_mult = trans2.split(';')
print(enr2.res2d.Term[index2])
print('total genes:',len(genes_mult))

DNA replication Homo sapiens hsa03030
total genes: 12


In [13]:
bool_df = mult_sig_df.Proteomics.isin(genes_mult)
plot_df2 = mult_sig_df[bool_df]
plot_df2.Proteomics.unique()

array(['MCM6', 'MCM4', 'PCNA', 'RFC5', 'MCM5', 'MCM2', 'MCM7', 'MCM3',
       'RFC3', 'RFC2', 'RFC4', 'RFC1'], dtype=object)

In [14]:
p.plotCircleHeatMap(plot_df2, circle_var = 'P_Value', color_var='Medians', x_axis= 'Proteomics', y_axis = 'Cancer',
                    plot_height=400, plot_width=800)

C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\plot_utils\__init__.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["size2"] = df[circle_var].apply(lambda x: -1*(np.log(x)))
C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\plot_utils\__init__.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['size'] = (df["size2"])*3


Loading BokehJS ...

In [15]:
get = pos_neg_df.Proteomics.isin(genes_mult) # bool df where True has both pos and neg
genes_pn_mult = pos_neg_df[get] # Keep only genes with pos and neg
genes_pn_mult.Proteomics.unique()

array(['MCM6', 'MCM4', 'RFC5', 'MCM5', 'MCM2', 'MCM7', 'MCM3', 'RFC2',
       'RFC4'], dtype=object)

In [16]:
p.plotCircleHeatMap(genes_pn_mult, circle_var = 'P_Value', color_var='Medians', x_axis= 'Proteomics', y_axis = 'Cancer',
                    plot_height=400, plot_width=600)

Loading BokehJS ...

# Check other pathways the genes (+/- and sig in mult. cancers) are part of

In [17]:
gnm = list(genes_pn_mult.Proteomics.unique())
print(gnm)
enr3 = gp.enrichr(gene_list = gnm, description='Tumor_partition', gene_sets='KEGG_2016', 
                       outdir='/Enrichr')

['MCM6', 'MCM4', 'RFC5', 'MCM5', 'MCM2', 'MCM7', 'MCM3', 'RFC2', 'RFC4']


In [18]:
enr3.res2d.head()

,Term,Overlap,P-value,Adjusted P-value,Old P-value,Old Adjusted P-value,Odds Ratio,Combined Score,Genes,Gene_set
0,DNA replication Homo sapiens hsa03030,9/36,6.673726e-26,1.955402e-23,0,0,555.555556,32205.018967,RFC5;RFC4;MCM7;RFC2;MCM3;MCM4;MCM5;MCM6;MCM2,KEGG_2016
1,Cell cycle Homo sapiens hsa04110,6/124,4.157916e-12,6.091346e-10,0,0,107.526882,2817.850237,MCM7;MCM3;MCM4;MCM5;MCM6;MCM2,KEGG_2016
2,Mismatch repair Homo sapiens hsa03430,3/23,1.110749e-07,1.084831e-05,0,0,289.855072,4641.467092,RFC5;RFC4;RFC2,KEGG_2016
3,Nucleotide excision repair Homo sapiens hsa03420,3/47,1.011533e-06,7.409480e-05,0,0,141.843972,1958.020353,RFC5;RFC4;RFC2,KEGG_2016


In [24]:
# Look at p-values 
certain_gene = 'PCNA'
all_df = pd.read_csv(root+R'\all_heatmap.csv')
gene_df = all_df.loc[all_df['Proteomics'] == certain_gene]
gene_df

,Proteomics,P_Value,Medians,Cancer
131,PCNA,0.000019,0.631486,Gbm
11449,PCNA,0.000844,0.420740,Hnscc
23452,PCNA,0.009321,0.501850,Luad
33458,PCNA,0.001106,0.870200,Lscc
46289,PCNA,0.139751,0.361150,Brca
58029,PCNA,0.413633,0.028375,Ov
64982,PCNA,0.012305,-0.235000,En
75182,PCNA,0.027668,-0.142000,Colon
